# INSTALL REQUIREMENTS

In [1]:
!pip install transformers

In [2]:
# for XLNet

!pip install sentencepiece

In [3]:
!pip install wandb

# GOOGLE MOUNT

In [4]:
from google.colab import drive
drive.mount('/content/drive')
PATH = './drive/MyDrive/datasets/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/groom_project1/

/content/drive/MyDrive/groom_project1


In [6]:
%pwd

'/content/drive/MyDrive/groom_project1'

# IMPORT REQUIREMENTS

In [7]:
import os
import sys
import random
import pickle

import numpy as np
from tqdm import tqdm

import torch
from torch.nn.utils.rnn import pad_sequence

from transformers import (
    AdamW,
    Adafactor,
)
from transformers.optimization import get_cosine_schedule_with_warmup

import wandb

# import costumized modules
from compute import compute_acc
from visualize_score import plot_graph
from dump_datasets import mk_dataset, mk_dataset_xlnet
from dump_models import load_model, load_model_xlnet
from evaluate import test_model
from data_processing import regular

#CREATE FOLDER

In [8]:
# create required folders if not exists

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder('./best_models')
createFolder('./dump_datasets')
createFolder('./dump_models_tokenizer')
createFolder('./scores')
createFolder('./submissions')

# FIX SEED

In [9]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

# MODEL

In [10]:
# test various models

MODEL_NAME = 'bert-base-uncased'
# MODEL_NAME = 'bert-large-uncased'
# MODEL_NAME = 'xlnet-base-cased'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
assert str(device) == 'cuda'


# by using pickle, load dump file if exist, else make dump file and load
# load for bert(uncased) or xlnet model(cased)

try:
    with open('./dump_models_tokenizer/' + MODEL_NAME + '.p', 'rb') as f:
        model = pickle.load(f)
        tokenizer = pickle.load(f)
        print('./dump_models_tokenizer/' + MODEL_NAME + '.p')
    print('model exists => just load model')
except:
    print('exeption occur => download model')
    if MODEL_NAME == 'bert-base-uncased':
        model, tokenizer = load_model(MODEL_NAME)
    elif MODEL_NAME == 'xlnet-base-cased':
        model, tokenizer = load_model_xlnet(MODEL_NAME)

model.to(device)

./dump_models_tokenizer/bert-base-uncased.p
model exists => just load model


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# HYPER PARAMETERS

In [11]:
TRAIN_BATCH_SIZE=256
EVAL_BATCH_SIZE=256
TRAIN_EPOCH = 3

LEARNING_RATE = 5e-5
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
# optimizer = Adafactor(model.parameters(), lr=LEARNING_RATE,
#                     eps=(1e-30, 1e-3), decay_rate=-0.8, clip_threshold=1.0,
#                     beta1=None,
#                     weight_decay=0.0,
#                     relative_step=False,
#                     scale_parameter=False,
#                     warmup_init=False,)
# scheduler = get_cosine_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=int(1732*TRAIN_EPOCH*0.1),
#                                             num_training_steps=1732*TRAIN_EPOCH)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# WANDB

In [12]:
!wandb login

wandb: Currently logged in as: chohs1221 (use `wandb login --relogin` to force relogin)


In [13]:
# wandb.init(project="test-project", entity="goorm_team_2")
wandb.init(project="my-test-project", entity="chohs1221")

wandb: Currently logged in as: chohs1221 (use `wandb login --relogin` to force relogin)


In [14]:
RUNNAME = 'realfinal_bert_ep3_' + str(LEARNING_RATE)
wandb.run.name = RUNNAME
wandb.config.learning_rate = LEARNING_RATE
wandb.config.epochs = TRAIN_EPOCH
wandb.config.batch_size = TRAIN_BATCH_SIZE

# LOAD DATASETS

In [15]:
# by using pickle, load dump file if exist, else make dump file and load
# load for bert(uncased) or xlnet model(cased)

try:
    if  MODEL_NAME == 'bert-base-uncased':
        with open('./dump_datasets/train_dev_dumps.p', 'rb') as f:
            train_pos = pickle.load(f)
            train_neg = pickle.load(f)
            dev_pos = pickle.load(f)
            dev_neg = pickle.load(f)
        print('dataset exists => just load datasets')
    elif  MODEL_NAME == 'xlnet-base-cased':
        with open('./dump_datasets/train_dev_dumps_xlnet.p', 'rb') as f:
            train_pos = pickle.load(f)
            train_neg = pickle.load(f)
            dev_pos = pickle.load(f)
            dev_neg = pickle.load(f)
        print('dataset exists => just load datasets')
except:
    print('exeption occur => make datasets')
    train_pos, train_neg, dev_pos, dev_neg = mk_dataset()
    if MODEL_NAME == 'bert-base-uncased':
        train_pos, train_neg, dev_pos, dev_neg = mk_dataset()
    elif MODEL_NAME == 'xlnet-base-cased':
        train_pos, train_neg, dev_pos, dev_neg = mk_dataset_xlnet()

dataset exists => just load datasets


# DATA PREPROCESSING

In [16]:
dev_neg[:10]

["windows have n't been cleaned in years you can see scum on them .\n",
 'waitresses are slow .\n',
 'just a mess avoid at all costs !\n',
 'bad !\n',
 'now pizza is beyond awful and wings are down there with its level .\n',
 'walked out of this place after _num_ min of no service .\n',
 "the place was n't even busy at this time .\n",
 'never will i be back to this place .\n',
 'this place is awful !\n',
 'not the food but the service .\n']

In [17]:
# Remove '_num_', special symbols(!@#%$% ..) in datasets

# train_pos = regular(train_pos)
# train_neg = regular(train_neg)
# dev_pos = regular(dev_pos)
# dev_neg = regular(dev_neg)

In [18]:
dev_neg[:10]

["windows have n't been cleaned in years you can see scum on them .\n",
 'waitresses are slow .\n',
 'just a mess avoid at all costs !\n',
 'bad !\n',
 'now pizza is beyond awful and wings are down there with its level .\n',
 'walked out of this place after _num_ min of no service .\n',
 "the place was n't even busy at this time .\n",
 'never will i be back to this place .\n',
 'this place is awful !\n',
 'not the food but the service .\n']

In [19]:
# use train, valid datasets for training

# train_pos = train_pos + dev_pos
# train_neg = train_neg + dev_neg
# len(train_pos)

# TOKENIZE

In [20]:
# seperate encoding to preprocess data before encoding

train_pos = [tokenizer.encode(line) for line in train_pos]
train_neg = [tokenizer.encode(line) for line in train_neg]
dev_pos = [tokenizer.encode(line) for line in dev_pos]
dev_neg = [tokenizer.encode(line) for line in dev_neg]

# MAKE DATASETS

In [21]:
# concatenate pos, neg dataset, costomize magic mathod

class SentimentDataset(object):
    def __init__(self, pos, neg):
        self.data = [pos_sent for pos_sent in pos] + [neg_sent for neg_sent in neg]
        self.label = [[1] for _ in range(len(pos))] + [[0] for _ in range(len(neg))]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

train_dataset = SentimentDataset(train_pos, train_neg)
dev_dataset = SentimentDataset(dev_pos, dev_neg)

# DATA LOADER

In [22]:
# argsort deleted <- no reason to sort
# attention masking for padding token

def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)

    attention_mask = torch.tensor([[1] * len(input_id) + [0] * (max_len - len(input_id)) for input_id in input_ids])
    input_ids = pad_sequence([torch.tensor(input_id) for input_id in input_ids], batch_first=True)
    token_type_ids = torch.tensor([[0] * len(input_id) for input_id in input_ids])
    position_ids = torch.tensor([list(range(len(input_id))) for input_id in input_ids])
    labels = torch.tensor(np.stack(labels, axis=0))

    return input_ids, attention_mask, token_type_ids, position_ids, labels

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=TRAIN_BATCH_SIZE,
                                           shuffle=True, 
                                           collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)

dev_loader = torch.utils.data.DataLoader(dev_dataset, 
                                         batch_size=EVAL_BATCH_SIZE,
                                         shuffle=False, 
                                         collate_fn=collate_fn_style,
                                         num_workers=2)

In [23]:
len(train_loader)

1732

# TRAIN

In [24]:
# record datas for highest accuracy & lowest loss

lowest_valid_loss = 9999.
highest_valid_acc = 0.
train_acc = []
train_loss = []
valid_acc = []
valid_loss = []

temp_train_acc = []
temp_train_loss = []

#train model

model.train()
for epoch in range(TRAIN_EPOCH):
    with tqdm(train_loader, unit="batch") as tepoch:
        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):

            tepoch.set_description(f"Epoch {epoch}")

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            output = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids,
                           position_ids=position_ids,
                           labels=labels)

            loss = output.loss
            
            logits = output.logits
            batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
            batch_labels = [int(example) for example in labels]
            
            acc = compute_acc(batch_predictions, batch_labels)
            
            loss.backward()
            optimizer.step()
            # scheduler.step()
            optimizer.zero_grad()


            tepoch.set_postfix(acc=acc, loss=loss.item())
            
            temp_train_acc.append(acc)
            temp_train_loss.append(loss.item())
            if iteration != 0 and iteration % int(len(train_loader) / 100) == 0:

                # evaluate model
                model.eval()
                with torch.no_grad():
                    val_acc = []
                    val_loss = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in dev_loader:
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    token_type_ids=token_type_ids,
                                    position_ids=position_ids,
                                    labels=labels)

                        logits = output.logits
                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        val_acc.append(compute_acc(batch_predictions, batch_labels))
                        val_loss.append(output.loss)


                # compute accuracy, loss for train, valid datasets and record
                mean_train_acc = sum(temp_train_acc) / len(temp_train_acc)
                mean_train_loss = sum(temp_train_loss) / len(temp_train_loss)
                mean_val_acc = sum(val_acc) / len(val_acc)
                mean_val_loss = sum(val_loss) / len(val_loss)

                train_acc.append(mean_train_acc)
                train_loss.append(mean_train_loss)
                valid_acc.append(mean_val_acc)
                valid_loss.append(mean_val_loss)

                temp_train_acc = []
                temp_train_loss = []
                

                # WANDB
                wandb.log({"train_loss": mean_train_loss,
                           'train_acc': mean_train_acc,
                           'valid_loss': mean_val_loss,
                           'valid_acc': mean_val_acc})


                # save best models
                if highest_valid_acc < mean_val_acc:
                    highest_valid_acc = mean_val_acc
                    print('ACCURACY for highest valid acc: ', mean_val_acc)
                    print('LOSS for lowest valid acc: ', mean_val_loss)
                    model.save_pretrained('./best_models/model' + str(int(mean_val_acc*100)) + str(int(mean_val_loss*1000)) + RUNNAME)

                elif lowest_valid_loss > mean_val_loss:
                    lowest_valid_loss = mean_val_loss
                    print('ACCURACY for lowest valid loss: ', mean_val_acc)
                    print('LOSS for lowest valid loss: ', mean_val_loss)
                    model.save_pretrained('./best_models/model' + str(int(mean_val_acc*100)) + str(int(mean_val_loss*1000)) + RUNNAME)
                                        
                model.train()
                                        
                model.train()
model.save_pretrained('./best_models/model' + str(int(mean_val_acc*100)) + str(int(mean_val_loss*1000)) + RUNNAME)

Epoch 0:   1%|          | 18/1732 [00:12<40:43,  1.43s/batch, acc=0.965, loss=0.102]

ACCURACY for highest valid acc:  0.956884765625
LOSS for lowest valid acc:  tensor(0.1374, device='cuda:0')


Epoch 0:   2%|▏         | 35/1732 [00:24<39:58,  1.41s/batch, acc=0.945, loss=0.157]

ACCURACY for highest valid acc:  0.959423828125
LOSS for lowest valid acc:  tensor(0.1136, device='cuda:0')


Epoch 0:   3%|▎         | 52/1732 [00:36<39:41,  1.42s/batch, acc=0.984, loss=0.0618]

...

ACCURACY for highest valid acc:  0.978955078125
LOSS for lowest valid acc:  tensor(0.0584, device='cuda:0')


Epoch 0:  62%|██████▏   | 1072/1732 [12:31<15:41,  1.43s/batch, acc=0.965, loss=0.0846]

ACCURACY for highest valid acc:  0.97900390625
LOSS for lowest valid acc:  tensor(0.0563, device='cuda:0')


Epoch 0:  66%|██████▌   | 1139/1732 [13:16<06:54,  1.43batch/s, acc=0.977, loss=0.0429]


KeyboardInterrupt: ignored

# SAVE SCORES

In [ ]:
# using pickle, save dump accuracy, loss file to use later when visualizing scores

accloss_filename = 'accloss' + str(int(mean_train_acc*100)) + str(int(mean_train_loss*1000)) + RUNNAME + '.p'
with open('./scores/' + accloss_filename,'wb') as f:
    pickle.dump(train_acc, f)
    pickle.dump(train_loss, f)
    pickle.dump(valid_acc, f)
    pickle.dump(valid_loss, f)

# TEST

In [ ]:
# test model

predictions = test_model(model, tokenizer, mean_val_acc, mean_val_loss, file_name = 'test_no_label', device='cuda')

# SCORE VISUALIZE

In [ ]:
# plot accuracy, loss graph for train, valid datasets from saved score pickle data

plot_graph(accloss_filename)